In [33]:
path_files = !ls data/*/*Annotated*txt
paths = !ls data/*/*Path*txt
path_files = set(path_files)
paths = set(paths)
paths = paths - path_files
print(paths)

vocab = set()
for level in path_files:
    with open(level) as level_file:
        vocab |= set(level_file.read().rstrip())

print(vocab)

level2path = {}
for level in path_files:
    path = level.replace('_Annotated','')
    with open(path) as path_file:
        path = []
        for line in path_file:
            path.append(eval(line.rstrip()))
    level2path[level] = path
    

{'data/MegaMan/megaman_1_2_Path.txt', 'data/MegaMan/megaman_1_5_Path.txt', 'data/Metroid/metroid_5_Path_2.txt', 'data/Metroid/metroid_5_Path_1.txt', 'data/Ninja Gaiden/NG_7_Path.txt', 'data/MegaMan/megaman_1_7_Path.txt', 'data/MegaMan/megaman_1_9_Path.txt', 'data/Mario/SuperMarioBros2(J)-WorldA-3_Path.txt', 'data/Ninja Gaiden/NG_4_Path.txt', 'data/Mario/mario-6-2_Path.txt', 'data/MegaMan/megaman_1_6_Path.txt', 'data/Mario/SuperMarioBros2(J)-WorldA-1_Path.txt', 'data/Castlevania/CV_2_Path.txt', 'data/Mario/SuperMarioBros2(J)-World3-3_Path.txt', 'data/Mario/mario-4-2_Path.txt', 'data/Mario/mario-1-1_Path.txt', 'data/Ninja Gaiden/NG_6_Path.txt', 'data/Mario/SuperMarioBros2(J)-World5-2_Path.txt', 'data/MegaMan/megaman_1_3_Path.txt', 'data/Ninja Gaiden/NG_2_Path.txt', 'data/Mario/SuperMarioBros2(J)-World2-3_Path.txt', 'data/Mario/SuperMarioBros2(J)-World4-1_Path.txt', 'data/Mario/SuperMarioBros2(J)-WorldD-1_Path.txt', 'data/Mario/mario-6-3_Path.txt', 'data/Metroid/metroid_4_Path_3.txt', 'da

In [34]:
import json
jsons = !ls data/*/*tile*json
print(jsons)
known = set(['-','x','\n'])
per_game = {'data/Castlevania/CV_tile_spec.json':{'collectable':['H','O','1','2','3','4','5','6','7','F','#','S','G','K','R','>','0','A','U'],
                                             'powerup':['1','F','#','S','5','H','G','O','>','A','U'],
                                             'breakable':['B'],
                                             'solid':['B'],
                                             'hazard':['E'],
                                             'moving':['E'],
                                             'null':['@'],
                                             'portal':['D']},
            'data/Kid Icarus - current agent/KI_tile_specs.json':{},
            'data/Mario/SMB_tile_specs.json':{'collectable':['o','Q'],
                                         'powerup':['?'],
                                         'hazard':['E','B'],
                                         'moving':['E'],
                                         'breakable':["S"]},
            'data/MegaMan/MM_tile_specs.json':{'null': ['@'], 
                                          'solid': ['#', 'B', 'H', 'M', 'C', 'D'], 
                                          'breakable': ['B'], 
                                          'hazard': ['H'], 
                                          'climbable': ['|'], 
                                          'collectable': ['L', 'l', 'W', 'w', '+', '*', 'U'],
                                          'powerup': ['L', 'l', 'W', 'w', '+', '*', 'U'], 
                                          'moving': ['M', 'D'],
                                          'portal':['t']},
            'data/Metroid/MT_tile_specs.json':{'breakable': ['B'],
                                          'solid':['B','[',']'],
                                          'portal':['D','(',')',],
                                          'collectable': ['+'],
                                          'powerup': ['+'], 
                                          'null':['@'],
                                          'hazard':['E','[',']'],
                                          'moving':['E']},
            'data/Ninja Gaiden/NG_tile_specs.json':{
                                            'hazard':['K','B','E','D','C'],
                                            'moving':['K','B','E','D','C'],
                                            'collectable': ['F','%',')','*','W','A','1','+','J','T','R'],
                                            'powerup': ['F','%',')','*','W','A','1','+','J','T'], }
}


per_game_annotations = {}

for j in jsons:
    tiles = json.load(open(j))
    tile2annotation = {}
    for tile_type in tiles:        
        for tile in tiles[tile_type]:            
            if tile not in tile2annotation:
                tile2annotation[tile] = []                
            tile2annotation[tile].append(tile_type)
    for tile_type in per_game[j]:        
        for tile in per_game[j][tile_type]:            
            if tile not in tile2annotation:
                tile2annotation[tile] = []                
            tile2annotation[tile].append(tile_type)
    per_game_annotations[j] = tile2annotation
    annotated = set(tile2annotation.keys())
    for level in path_files:
        if level[:8] == j[:8]:
            with open(level) as level_file:
                level_str = level_file.read()
                level_vocab = set(level_str)
                missing = (level_vocab-annotated)-known
                if len(missing) > 0:
                    print(missing)
                    print(j,level)
                    
ignored = set(['passable'])  
combos = set()
for game in per_game_annotations:
    for tile in per_game_annotations[game]:
        affordances = tuple(sorted(set(per_game_annotations[game][tile])-ignored))
        combos.add(affordances)

common_vocab = {('breakable', 'solid'):'S', 
                ('climbable', 'solid'):'|', 
                ('collectable',):'o', 
                ('collectable', 'powerup'):'*', 
                ('collectable', 'solid'):'Q', 
                ('hazard',):'v', 
                ('hazard', 'moving'):'e', 
                ('hazard', 'moving', 'solid'):'E', 
                ('hazard', 'null', 'solid'):'@', 
                ('hazard', 'solid'):'^', 
                ('moving', 'solid'):'#', 
                ('portal',):'$',  
                ('powerup', 'solid'):'!',
                ('solid',):'X'}

print(len(combos-set(common_vocab.keys())))

import os
for j in jsons:
    game_folder = '/'.join(j.split('/')[:-1])
    try:
        os.mkdir(f'{game_folder}/affordances/')
    except:
        pass
    
    for level in path_files:
        if level[:8] == j[:8]:
            with open(level) as level_file:
                
                level_str = level_file.read()
                affordance_str = ''
                for t in level_str:
                    tok = '-'
                    if t in set(['x','-','\n']):
                        tok = t
                    elif t in per_game_annotations[j]:
                        affordances = tuple(sorted(set(per_game_annotations[j].get(t))-ignored))
                        tok = common_vocab[affordances]
                    affordance_str += tok
            level_name = level.split('/')[-1]
            with open(f'{game_folder}/affordances/{level_name}','w') as outfile:
                outfile.write(affordance_str)
                

['data/Castlevania/CV_tile_spec.json', 'data/Mario/SMB_tile_specs.json', 'data/MegaMan/MM_tile_specs.json', 'data/Metroid/MT_tile_specs.json', 'data/Ninja Gaiden/NG_tile_specs.json']
{'P'}
data/Castlevania/CV_tile_spec.json data/Castlevania/CV_1_Annotated_Path.txt
{'P'}
data/MegaMan/MM_tile_specs.json data/MegaMan/megaman_1_3_Annotated_Path.txt
{'P'}
data/Ninja Gaiden/NG_tile_specs.json data/Ninja Gaiden/NG_6_Annotated_Path.txt
{'P'}
data/Ninja Gaiden/NG_tile_specs.json data/Ninja Gaiden/NG_2_Annotated_Path.txt
{'P'}
data/Ninja Gaiden/NG_tile_specs.json data/Ninja Gaiden/NG_5_Annotated_Path.txt
{'P'}
data/Ninja Gaiden/NG_tile_specs.json data/Ninja Gaiden/NG_4_Annotated_Path.txt
0


In [15]:
T = {
        "@" : ["null"],
        "#" : ["solid","ground"],
        "B" : ["solid","breakable"],
        "H" : ["solid", "hazard"],
        "|" : ["passable", "climbable"],
        "L" : ["passable", "collectable", "powerup"],
        "l" : ["passable", "collectable", "powerup"],
        "W" : ["passable", "collectable", "powerup"],
        "w" : ["passable", "collectable", "powerup"],
        "+" : ["passable", "collectable", "powerup"],
        "*" : ["passable", "collectable", "powerup"],
        "U" : ["passable", "collectable", "powerup"],
        "M" : ["solid", "moving"],
        "P" : ["passable", "empty"],
        "C" : ["solid", "cannon"],
        "D" : ["solid", "moving"],
        "t" : ["passable", "empty"]
}

In [16]:
tile_types = {}
for tile in T:
    for ty in T[tile]:
        if ty not in tile_types:
            tile_types[ty] = []
        tile_types[ty].append(tile)
print(tile_types)

{'null': ['@'], 'solid': ['#', 'B', 'H', 'M', 'C', 'D'], 'ground': ['#'], 'breakable': ['B'], 'hazard': ['H'], 'passable': ['|', 'L', 'l', 'W', 'w', '+', '*', 'U', 'P', 't'], 'climbable': ['|'], 'collectable': ['L', 'l', 'W', 'w', '+', '*', 'U'], 'powerup': ['L', 'l', 'W', 'w', '+', '*', 'U'], 'moving': ['M', 'D'], 'empty': ['P', 't'], 'cannon': ['C']}
